> Please go through Giba's post and kernel  to underrstand what this leak is all about
> https://www.kaggle.com/titericz/the-property-by-giba (kernel)
> https://www.kaggle.com/c/santander-value-prediction-challenge/discussion/61329 (post)
> 
> Also, go through this Jiazhen's kernel which finds more columns to exploit leak
> https://www.kaggle.com/johnfarrell/giba-s-property-extended-result
> 
> I just exploit data property in brute force way and then fill in remaining by row non zero means! This should bring everyone on level-playing field.
> 
> **Let the competition begin! :D**

### Just some small modifications from [original baseline](https://www.kaggle.com/tezdhar/breaking-lb-fresh-start)~
- The leak rows are calculated separately on train/test set
- Calculated the leaky values, correctness, for each lag
- Hope this can help to do some *lag_selection*

### Update leak process codes to Dmitry Frumkin's *fast* [version](https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2)
- The result of Dmitry's original function and result of Hasan's function seem slightly different
- Modified to make the output consistent with Hasan's function (Seems better score)

In [29]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

# Any results you write to the current directory are saved as output.

In [30]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

transact_cols = [f for f in train.columns if f not in ["ID", "target"]]
y = np.log1p(train["target"]).values

In [5]:
test["target"] = train["target"].mean()

We take time series columns from [here](https://www.kaggle.com/johnfarrell/giba-s-property-extended-result)

In [38]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]

# cols = ['f3cf9341c','fa11da6df','d47c58fe2','555f18bd3','134ac90df','716e7d74d','1bf8c2597',
#                '1f6b2bafa','174edf08a','5bc7ab64f','a61aa00b0','b2e82c050','26417dec4',
#                '51707c671','e8d9394a0','cbbc9c431','6b119d8ce','f296082ec','be2e15279',
#                '698d05d29','38e6f8d32','93ca30057','7af000ac2','1fd0a1f2a','41bc25fef',
#                '0df1d7b9a','2b2b5187e','bf59c51c3','cfe749e26','ad207f7bb','11114a47a',
#                'a8dd5cea5','b88e5de84']

In [33]:
# from: https://www.kaggle.com/dfrumkin/a-simple-way-to-use-giba-s-features-v2
def _get_leak(df, cols, lag=0):
    d1 = df[cols[:-lag-2]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2 = df[cols[lag+2:]].apply(tuple, axis=1).to_frame().rename(columns={0: 'key'})
    d2['pred'] = df[cols[lag]]
    #d2 = d2[d2.pred != 0] ### to make output consistent with Hasan's function
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    return d1.merge(d3, how='left', on='key').pred.fillna(0)

In [34]:
def compiled_leak_result():
    
    max_nlags = len(cols) - 2
    train_leak = train[["ID", "target"] + cols]
    train_leak["compiled_leak"] = 0
    train_leak["nonzero_mean"] = train[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        train_leak[c] = _get_leak(train_leak, cols, i)
        
        leaky_cols.append(c)
        train_leak = train.join(
            train_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = train_leak["compiled_leak"]==0
        train_leak.loc[zeroleak, "compiled_leak"] = train_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(train_leak["compiled_leak"] > 0))
        _correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in train", leaky_value_counts[-1])
        print(
            "% of correct leaks values in train ", 
            leaky_value_corrects[-1]
        )
        tmp = train_leak.copy()
        tmp.loc[zeroleak, "compiled_leak"] = train_leak["nonzero_mean"]
        scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        print(
            'Score (filled with nonzero mean)', 
            scores[-1]
        )
    result = dict(
        score=scores, 
        leaky_count=leaky_value_counts,
        leaky_correct=leaky_value_corrects,
    )
    return train_leak, result

In [39]:
train_leak, result = compiled_leak_result()

Processing lag 0
Leak values found in train 299
% of correct leaks values in train  0.979933110367893
Score (filled with nonzero mean) 1.5138333391635181
Processing lag 1
Leak values found in train 446
% of correct leaks values in train  0.984304932735426
Score (filled with nonzero mean) 1.4772272652360527
Processing lag 2
Leak values found in train 565
% of correct leaks values in train  0.9787610619469026
Score (filled with nonzero mean) 1.4609901689108993
Processing lag 3
Leak values found in train 629
% of correct leaks values in train  0.9809220985691574
Score (filled with nonzero mean) 1.4467458096662653
Processing lag 4
Leak values found in train 670
% of correct leaks values in train  0.9791044776119403
Score (filled with nonzero mean) 1.4369522015753753
Processing lag 5
Leak values found in train 705
% of correct leaks values in train  0.9773049645390071
Score (filled with nonzero mean) 1.429888414041253
Processing lag 6
Leak values found in train 735
% of correct leaks values

KeyboardInterrupt: 

In [ ]:
result = pd.DataFrame.from_dict(result, orient='columns')
result.T

In [ ]:
result.to_csv('train_leaky_stat.csv', index=False)

In [ ]:
train_leak.head()

In [ ]:
best_score = np.min(result['score'])
best_lag = np.argmin(result['score'])
print('best_score', best_score, '\nbest_lag', best_lag)

In [ ]:
def rewrite_compiled_leak(leak_df, lag):
    leak_df["compiled_leak"] = 0
    for i in range(lag):
        c = "leaked_target_"+str(i)
        zeroleak = leak_df["compiled_leak"]==0
        leak_df.loc[zeroleak, "compiled_leak"] = leak_df.loc[zeroleak, c]
    return leak_df

In [ ]:
leaky_cols = [c for c in train_leak.columns if 'leaked_target_' in c]
train_leak = rewrite_compiled_leak(train_leak, best_lag)
train_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
train_res = train_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
train_res.to_csv('train_leak.csv', index=False)

In [ ]:
def compiled_leak_result_test(max_nlags):
    test_leak = test[["ID", "target"] + cols]
    test_leak["compiled_leak"] = 0
    test_leak["nonzero_mean"] = test[transact_cols].apply(
        lambda x: np.expm1(np.log1p(x[x!=0]).mean()), axis=1
    )
    
    scores = []
    leaky_value_counts = []
    # leaky_value_corrects = []
    leaky_cols = []
    
    for i in range(max_nlags):
        c = "leaked_target_"+str(i)
        
        print('Processing lag', i)
        test_leak[c] = _get_leak(test_leak, cols, i)
        
        leaky_cols.append(c)
        test_leak = test.join(
            test_leak.set_index("ID")[leaky_cols+["compiled_leak", "nonzero_mean"]], 
            on="ID", how="left"
        )[["ID", "target"] + cols + leaky_cols+["compiled_leak", "nonzero_mean"]]
        zeroleak = test_leak["compiled_leak"]==0
        test_leak.loc[zeroleak, "compiled_leak"] = test_leak.loc[zeroleak, c]
        leaky_value_counts.append(sum(test_leak["compiled_leak"] > 0))
        #_correct_counts = sum(train_leak["compiled_leak"]==train_leak["target"])
        #leaky_value_corrects.append(_correct_counts/leaky_value_counts[-1])
        print("Leak values found in test", leaky_value_counts[-1])
        #print(
        #    "% of correct leaks values in train ", 
        #    leaky_value_corrects[-1]
        #)
        #tmp = train_leak.copy()
        #tmp.loc[zeroleak, "compiled_leak"] = tmp.loc[zeroleak, "nonzero_mean"]
        #scores.append(np.sqrt(mean_squared_error(y, np.log1p(tmp["compiled_leak"]).fillna(14.49))))
        #print(
        #    'Score (filled with nonzero mean)', 
        #    scores[-1]
        #)
    result = dict(
        # score=scores, 
        leaky_count=leaky_value_counts,
        # leaky_correct=leaky_value_corrects,
    )
    return test_leak, result

In [ ]:
test_leak, test_result = compiled_leak_result_test(max_nlags=38)

In [ ]:
test_result = pd.DataFrame.from_dict(test_result, orient='columns')
test_result.T

In [ ]:
test_result.to_csv('test_leaky_stat.csv', index=False)

In [ ]:
test_leak = rewrite_compiled_leak(test_leak, best_lag)
test_leak[['ID']+leaky_cols+['compiled_leak']].head()

In [ ]:
test_res = test_leak[leaky_cols+['compiled_leak']].replace(0.0, np.nan)
test_res.to_csv('test_leak.csv', index=False)

In [ ]:
# test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = test_leak.loc[test_leak["compiled_leak"]==0, "nonzero_mean"]
test_leak.loc[test_leak["compiled_leak"]==0, "compiled_leak"] = 0

In [ ]:
#submission
sub = test[["ID"]]
sub["target"] = test_leak["compiled_leak"]
sub.to_csv(f"baseline_sub_lag_{best_lag}.csv", index=False)
print(f"baseline_sub_lag_{best_lag}.csv saved")

In [ ]:
sub.head()